In [1]:
import pandas as pd

In [2]:
pesticides_data = pd.read_excel("dataset.xlsx", sheet_name="Original data", header=0)

# Remove any samples containing missing values, remove duplicates

pesticides_data = pesticides_data.dropna(axis=0, how="any")
idx = pesticides_data[pesticides_data["LC50 (mg/kg dry soil)"] == "-"].index
pesticides_data = pesticides_data.drop(index=idx).reset_index(drop=True)
pesticides_data = pesticides_data.applymap(
    lambda x: x.strip() if type(x) == str else x
)  ##  Remove spaces
pesticides_data = pesticides_data.drop_duplicates(keep="first", ignore_index=True)

# Removal of metal-organic compounds, salts and mixtures
# (prior to this, inorganic substances and some obvious mixtures had been removed manually)

elements = ["Fe", "Cu", "Zn", "Na", "K", "NH4+", "Al", "Mg", "Mn", "Pb", "Sn", "."]

idxes = []
for element in elements:
    for idx in pesticides_data.index.values:
        if element in pesticides_data.loc[idx, "SMILES"]:
            idxes.append(idx)
        else:
            pass

pesticides_data.drop(index=idxes, inplace=True)
pesticides_data.reset_index(drop=True, inplace=True)

# Screening for conflicting data
# i.e., data with the same SMILES, pesticide formulation and soil type but different LC50 values

conflict_data = pesticides_data[
    pesticides_data.duplicated(
        subset=["Pesticide formulation", "SMILES", "Soil type"], keep=False
    )
]

conflict_data = conflict_data.sort_values(
    by=["SMILES", "Pesticide formulation", "Soil type", "English name"],
    ignore_index=True,
)

# conflict_data.to_csv('conflict data.csv', index=False)  ##  Manual handling of conflicting data

In [ ]:
# Remove conflicting data and import manually processed conflicting data

pesticides_data = pesticides_data.drop_duplicates(
    subset=["SMILES", "Pesticide formulation", "Soil type"],
    keep=False,
    ignore_index=True,
)

conflict_data = pd.read_excel("dataset.xlsx", sheet_name="Conflict data", header=0)
pesticides_data = pd.concat([pesticides_data, conflict_data], axis=0, ignore_index=True)
pesticides_data.drop_duplicates(keep="first", inplace=True)  ##  ensure no duplicates

print(f"The type of pesticides are {len(set(pesticides_data['SMILES']))}.")
print(f"The type of soil is {len(set(pesticides_data['Soil type']))}.")
print(
    f"The type of pesticide formulations are {len(set(pesticides_data['Pesticide formulation']))}."
)

# Convert LC50 values to numeric values and classify them into different toxicity levels

pesticides_data["LC50 (mg/kg dry soil)"] = pesticides_data[
    "LC50 (mg/kg dry soil)"
].astype(str)

for idx in pesticides_data.index:
    if ">" in pesticides_data.loc[idx, "LC50 (mg/kg dry soil)"]:
        pesticides_data.loc[idx, "LC50 (mg/kg dry soil)"] = pesticides_data.loc[
            idx, "LC50 (mg/kg dry soil)"
        ].replace(">", "")
    if "<" in pesticides_data.loc[idx, "LC50 (mg/kg dry soil)"]:
        pesticides_data.loc[idx, "LC50 (mg/kg dry soil)"] = pesticides_data.loc[
            idx, "LC50 (mg/kg dry soil)"
        ].replace("<", "")

pesticides_data["Toxicity level"] = ""
pesticides_data["LC50 (mg/kg dry soil)"] = pesticides_data[
    "LC50 (mg/kg dry soil)"
].astype(float)

# Binary classification task

pesticides_data.loc[
    (pesticides_data["LC50 (mg/kg dry soil)"] < 100), "Toxicity level"
] = 1
pesticides_data.loc[
    (pesticides_data["LC50 (mg/kg dry soil)"] >= 100), "Toxicity level"
] = 0

# Three-class classification task

# pesticides_data.loc[(pesticides_data["LC50 (mg/kg dry soil)"] < 10), 'Toxicity level'] = 2
# pesticides_data.loc[(pesticides_data["LC50 (mg/kg dry soil)"] >= 10) & (pesticides_data["LC50 (mg/kg dry soil)"] < 100), 'Toxicity level'] = 1
# pesticides_data.loc[(pesticides_data["LC50 (mg/kg dry soil)"] >= 100), 'Toxicity level'] = 0

pesticides_data = pesticides_data.drop(columns=["LC50 (mg/kg dry soil)"])

# The soil types were converted to their corresponding pH and organic matter (OM) values,
# which were obtained from the data of the Second Soil Census of China as well as from the literatures

pesticides_data["pH"] = ""
pesticides_data["Organic matter"] = ""

for idx in pesticides_data.index.values:

    if pesticides_data.loc[idx, "Soil type"] == "Artificial soil":
        pesticides_data.loc[idx, "pH"] = 6
        pesticides_data.loc[idx, "Organic matter"] = 8.5

    elif pesticides_data.loc[idx, "Soil type"] == "Northeast black soil":
        pesticides_data.loc[idx, "pH"] = 6.78
        pesticides_data.loc[idx, "Organic matter"] = 30.41

    elif pesticides_data.loc[idx, "Soil type"] == "Taihu Lake paddy soil":
        pesticides_data.loc[idx, "pH"] = 6.12
        pesticides_data.loc[idx, "Organic matter"] = 30.88

    elif pesticides_data.loc[idx, "Soil type"] == "Wuxi paddy soil":
        pesticides_data.loc[idx, "pH"] = 6.96
        pesticides_data.loc[idx, "Organic matter"] = 23.52

    elif pesticides_data.loc[idx, "Soil type"] == "Henan Erhe soil":
        pesticides_data.loc[idx, "pH"] = 7.6
        pesticides_data.loc[idx, "Organic matter"] = 11.3

    elif pesticides_data.loc[idx, "Soil type"] == "Jiangxi red soil":
        pesticides_data.loc[idx, "pH"] = 5.1
        pesticides_data.loc[idx, "Organic matter"] = 23.15

    elif pesticides_data.loc[idx, "Soil type"] == "Meadow brown loam":
        pesticides_data.loc[idx, "pH"] = 6.06
        pesticides_data.loc[idx, "Organic matter"] = 19.7

    elif pesticides_data.loc[idx, "Soil type"] == "Hunan soil":
        pesticides_data.loc[idx, "pH"] = 4.57
        pesticides_data.loc[idx, "Organic matter"] = 0.78

    elif pesticides_data.loc[idx, "Soil type"] == "Shanxi soil":
        pesticides_data.loc[idx, "pH"] = 7.17
        pesticides_data.loc[idx, "Organic matter"] = 1.35

    elif pesticides_data.loc[idx, "Soil type"] == "Beijing soil":
        pesticides_data.loc[idx, "pH"] = 6.86
        pesticides_data.loc[idx, "Organic matter"] = 1.3

    elif pesticides_data.loc[idx, "Soil type"] == "Heilongjiang soil":
        pesticides_data.loc[idx, "pH"] = 6.35
        pesticides_data.loc[idx, "Organic matter"] = 2.26

    elif pesticides_data.loc[idx, "Soil type"] == "Jiangsu soil":
        pesticides_data.loc[idx, "pH"] = 5.8
        pesticides_data.loc[idx, "Organic matter"] = 2.84

    elif pesticides_data.loc[idx, "Soil type"] == "Changsha soil":
        pesticides_data.loc[idx, "pH"] = 4.24
        pesticides_data.loc[idx, "Organic matter"] = 4.73

    elif pesticides_data.loc[idx, "Soil type"] == "Guangzhou soil":
        pesticides_data.loc[idx, "pH"] = 7.23
        pesticides_data.loc[idx, "Organic matter"] = 5.45

    elif pesticides_data.loc[idx, "Soil type"] == "Hangzhou soil":
        pesticides_data.loc[idx, "pH"] = 5.15
        pesticides_data.loc[idx, "Organic matter"] = 5.58

    elif pesticides_data.loc[idx, "Soil type"] == "Zhejiang soil":
        pesticides_data.loc[idx, "pH"] = 5.18
        pesticides_data.loc[idx, "Organic matter"] = 1.60

    elif pesticides_data.loc[idx, "Soil type"] == "Taian soil":
        pesticides_data.loc[idx, "pH"] = 6.36
        pesticides_data.loc[idx, "Organic matter"] = 22.5

    elif pesticides_data.loc[idx, "Soil type"] == "Institution soil":
        pesticides_data.loc[idx, "pH"] = 4
        pesticides_data.loc[idx, "Organic matter"] = 2.71

pesticides_data = pesticides_data.drop(columns=["Soil type"])
pesticides_data["pH"] = pd.to_numeric(pesticides_data["pH"])
pesticides_data["Organic matter"] = pd.to_numeric(pesticides_data["Organic matter"])
pesticides_data["Toxicity level"] = pd.to_numeric(pesticides_data["Toxicity level"])

pesticides_data.sort_values(
    by=["SMILES", "Pesticide formulation", "Organic matter", "pH"],
    ascending=True,
    ignore_index=True,
    inplace=True,
)

pesticides_data = pesticides_data[
    [
        "English name",
        "CAS number",
        "SMILES",
        "Pesticide formulation",
        "pH",
        "Organic matter",
        "Toxicity level",
    ]
]
pesticides_data["English name"] = pesticides_data["English name"].str.title()

pesticides_data

# pesticides_data.to_csv("Binary classification.csv", index=False)
# pesticides_data.to_csv("Three-class classification.csv", index=False)

The type of pesticides are 200.
The type of soil is 17.
The type of pesticide formulations are 20.


,English name,CAS number,SMILES,Pesticide formulation,pH,Organic matter,Toxicity level
0,Flumioxazin,103361-09-7,C#CCN1C(=O)COC2=CC(=C(C=C21)N3C(=O)C4=C(C3=O)C...,WP,6.00,8.50,1
1,Chlorothalonil,1897-45-6,C(#N)C1=C(C(=C(C(=C1Cl)Cl)Cl)C#N)Cl,SC,6.00,8.50,0
2,Propiconazole,60207-90-1,C(C(=O)O)NCP(=O)(O)O,EC,6.00,8.50,0
3,Propiconazole,60207-90-1,C(C(=O)O)NCP(=O)(O)O,EC,6.78,30.41,0
4,Propiconazole,60207-90-1,C(C(=O)O)NCP(=O)(O)O,EC,6.12,30.88,0
...,...,...,...,...,...,...,...
568,Flusilazole,85509-19-9,C[Si](CN1C=NC=N1)(C2=CC=C(C=C2)F)C3=CC=C(C=C3)F,EW,6.12,30.88,0
569,Flusilazole,85509-19-9,C[Si](CN1C=NC=N1)(C2=CC=C(C=C2)F)C3=CC=C(C=C3)F,ME,6.00,8.50,0
570,Flusilazole,85509-19-9,C[Si](CN1C=NC=N1)(C2=CC=C(C=C2)F)C3=CC=C(C=C3)F,ME,6.78,30.41,0
571,Flusilazole,85509-19-9,C[Si](CN1C=NC=N1)(C2=CC=C(C=C2)F)C3=CC=C(C=C3)F,ME,6.12,30.88,0
